In [1]:
# magic reload
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')
from analysis.analysis_utils import *
from analysis.plot_utils import *
from datasets import load_dataset

### Accuracy of QA expert

In [ ]:
qa_df = prepare_df(types=['qa'])
filter = {'config_model_name_qa': 'deepseek/deepseek-v3.1-terminus', 'config_dataset_name_qa': 'Idavidrein/gpqa', 'config_dataset_subset_qa': 'gpqa_diamond', 'config_dataset_split_qa': 'train'}
qa_df_deepseek_31_terminus = qa_df[qa_df.apply(lambda row: all(row[col] == val for col, val in filter.items()), axis=1)]

(196, 30)

In [3]:
print(qa_df_deepseek_31_terminus['is_correct_qa'].mean().round(2))

0.77


### Analyze debate

In [5]:
debate_df = prepare_df(types=['debates'])
filter = {'run_id_debates': 'z1qae3w'}
debate_df_4_choice = debate_df[debate_df.apply(lambda row: all(row[col] == val for col, val in filter.items()), axis=1)]

In [ ]:
debate_df_4_choice['debate_history_debates'].apply(lambda x: sum_reasoning_tokens_over_turns(x)).value_counts()

debate_history_debates
0    154
Name: count, dtype: int64

In [8]:
tot_issues = 0
for idx, row in debate_df_4_choice.iterrows():
    debate_history = row['debate_history_debates']
    for turn in debate_history:
        public_argument = turn['parsed_response']['public_argument'].lower()
        if 'private' in public_argument:
            print('='*80)
            print(f"issue_type: private in argument, record_id: {row['record_id_debates']}, debater_idx: {turn['debater_idx']}, turn: {turn['turn']}, success: {row['success_debates']}")
            print('='*80)
            print(public_argument)
            tot_issues += 1
        elif 'my answer is wrong' in public_argument:
            print('='*80)
            print(f"issue_type: wrong in argument, record_id: {row['record_id_debates']}, debater_idx: {turn['debater_idx']}, turn: {turn['turn']}, success: {row['success_debates']}")
            print('='*80)
            print(public_argument)
            tot_issues += 1
print(f"\n\ntotal issues: {tot_issues}")


issue_type: private in argument, record_id: mm9546i, debater_idx: 3, turn: 0, success: 1.0
the correct interplanar distance for the (111) plane in a rhombohedral lattice is determined by the reciprocal lattice vector. for a rhombohedral crystal with a=b=c=10 å and α=β=γ=30°, the general formula for the d-spacing is derived from the metric tensor. the specific formula for the (111) plane simplifies to d_{111} = a / √(3 + 6\cosα(1 - \cosα) / (1 + 2\cosα)).

substituting the given values, a=10 and α=30° (cos30°=√3/2≈0.8660), the calculation proceeds as follows: the denominator is √(3 + 6*0.8660*(1-0.8660)/(1+2*0.8660)) = √(3 + 5.196*0.134/2.732) = √(3 + 0.696/2.732) ≈ √(3 + 0.255) = √3.255 ≈ 1.804. thus, d_{111} = 10 / 1.804 ≈ 5.54 å.

wait, this calculation gives 5.54 å, not 9.54 å. there is a critical error in my initial argument. i must recalculate carefully. the correct simplified formula for the (111) plane when a=b=c is d = a √( (1 - cosα) / (1 + 2cosα) ). using α=30°, cosα=√3/2, 1-